In [91]:
import pandas as pd
# pd.set_option("display.max_rows", None)
# pd.set_option("display.max_columns", None)
# pd.set_option("display.max_colwidth", None)
import yfinance as yf
from transformers import AutoTokenizer, AutoModelForCausalLM,pipeline
import torch
model_id = "tarun7r/Finance-Llama-8B"

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",             # auto-distributes across GPUs
    torch_dtype="auto",            # picks bf16 or fp16 depending on availability
)


Loading checkpoint shards: 100%|██████████| 4/4 [00:01<00:00,  2.49it/s]


In [92]:
def get_financial_info(t):
    ticker = yf.Ticker('t')
    income_stmt = ticker.financials
    balance_sheet = ticker.balance_sheet
    cash_flow = ticker.cashflow
    info = ticker.info
    year=income_stmt.columns[0]
    EPS=income_stmt[year]["Diluted EPS"]
    Net_Income=income_stmt[year]["Net Income"]
    Gross_Profit=income_stmt[year]["Gross Profit"]
    Revenue=income_stmt[year]["Total Revenue"]
    Total_Assets=balance_sheet[year]["Total Assets"]
    Total_Liabilities=balance_sheet[year]["Total Liabilities Net Minority Interest"]
    Shareholders_Equity=balance_sheet[year]["Stockholders Equity"]
    Free_Cash_Flow=cash_flow[year]["Free Cash Flow"]
    Investing_Cash_Flow=cash_flow[year]["Investing Cash Flow"]
    Financing_Cash_Flow=cash_flow[year]["Financing Cash Flow"]
    Operating_Cash_Flow=cash_flow[year]["Operating Cash Flow"]
    P_E=ticker.info['trailingPE']
    ROA=ticker.info['returnOnAssets']
    ROE=ticker.info['returnOnEquity']

    prompt = f"""
        You are a fundamental investment analyst. Analyze the financial performance of {t} and give a recommendation: Strong Buy, Buy, Hold, Sell, or Short. 
        Justify your decision in 4–6 bullet points using financial reasoning. Consider all the financial information shared. Only use the numerical data given. 
        Do not add assumptions about company operations, reputation, or strategy.


        Financials for {t}:

        Income Statement:
        - Revenue: ${Revenue:,.0f}
        - Gross Profit: ${Gross_Profit:,.0f}
        - Net Income: ${Net_Income:,.0f}
        - EPS (Diluted): {EPS:.2f}

        Balance Sheet:
        - Total Assets: ${Total_Assets:,.0f}
        - Total Liabilities: ${Total_Liabilities:,.0f}
        - Shareholders' Equity: ${Shareholders_Equity:,.0f}

        Cash Flow:
        - Operating Cash Flow: ${Operating_Cash_Flow:,.0f}
        - Free Cash Flow: ${Free_Cash_Flow:,.0f}
        - Investing Cash Flow: ${Investing_Cash_Flow:,.0f}
        - Financing Cash Flow: ${Financing_Cash_Flow:,.0f}

        Valuation and Ratios:
        - P/E Ratio: {P_E:.2f}
        - ROA: {ROA:.2%}
        - ROE: {ROE:.2%}

        Based on this, what is your investment recommendation? Pick one action candidate.
        """
    return prompt

In [93]:
prompt=get_financial_info("AAPL")
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer
)

outputs = generator(
    prompt,
    #max_new_tokens=250,         # Reduced for memory efficiency
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    pad_token_id=tokenizer.eos_token_id,
    # Memory efficient generation settings
    num_beams=1,                # No beam search to save memory
    #early_stopping=True,
    use_cache=True
)

generated_text = outputs[0]['generated_text']
print(generated_text)

Device set to use cuda:0



        You are a fundamental investment analyst. Analyze the financial performance of AAPL and give a recommendation: Strong Buy, Buy, Hold, Sell, or Short. 
        Justify your decision in 4–6 bullet points using financial reasoning. Consider all the financial information shared. Only use the numerical data given. 
        Do not add assumptions about company operations, reputation, or strategy.


        Financials for AAPL:

        Income Statement:
        - Revenue: $122,336,000,000
        - Gross Profit: $73,115,000,000
        - Net Income: $10,948,000,000
        - EPS (Diluted): 1.49

        Balance Sheet:
        - Total Assets: $394,795,000,000
        - Total Liabilities: $274,570,000,000
        - Shareholders' Equity: $104,372,000,000

        Cash Flow:
        - Operating Cash Flow: $38,771,000,000
        - Free Cash Flow: $18,508,000,000
        - Investing Cash Flow: $-17,490,000,000
        - Financing Cash Flow: $-24,708,000,000

        Valuation and Ratios: